In [1]:
from smartcard import ATR
from smartcard.System import readers
from smartcard.CardConnection import CardConnection

In [90]:
rarr=readers()
if len(rarr) == 0:
    print("No readers detected")
else:
    reader = rarr[0]
    print(reader)

Generic Smart Card Reader Interface


In [91]:
protocol = CardConnection.T1_protocol
connection = reader.createConnection()
connection.connect(protocol)

In [92]:
atr=bytes(connection.getATR())
atr.hex()

'3bdc18ff8191fe1fc38073c821136605036351000250'

In [93]:
def select(appletID):
    data = list(bytes.fromhex(appletID))
    # Select:
    # CLA = 0x00
    # INS = 0xA4
    # P1 = 0x04
    # P2 = 0x00
    # Data = the instance AID
    cmd = [0x00, # CLA
              0xA4, # INS
              0x04, # P1
              0x00, # P2
              len(data), # Lc (content length)
          ] + data + [0x00]
    print(bytes(cmd).hex().upper())
    data, *sw = connection.transmit(cmd,protocol)
    data = bytes(data)
    sw = bytes(sw)
    if sw == b"\x90\x00":
        return data
    else:
        raise RuntimeError("Card responded with code %s and data \"%s\"" % (sw.hex(), data.hex()))

In [94]:
select("B00B5111CB01")

00A4040006B00B5111CB0100


b''

In [95]:
def request(APDU):
    cmd = list(bytes.fromhex(APDU))
    data, *sw = connection.transmit(cmd,protocol)
    data = bytes(data)
    sw = bytes(sw)
    if sw == b"\x90\x00":
        return data
    else:
        raise RuntimeError("Card responded with code %s and data \"%s\"" % (sw.hex(), data.hex()))

In [116]:
# get pubkey - private key is generated when applet is installed
request("B0A1000000").hex()

'049f34fc8f0998444494593dbb427cff4317f32ba4c76f62eb3a006261ce4d6b75682be7045b4062fc73d8075f01a513b1b15056082dbaa61e6f75dd3d80bef775'

In [119]:
# get random data
rand = request("B0A2000000")
rand.hex()

'0383b8b5144d1cbfac80db734016018299e291aa7d53f6d0e2b8c9a2aa45e6c3'

In [122]:
# static data for any other APDU
data = request("B000000000")
data

b'Memory cards are not safu so what?'

In [123]:
connection.disconnect()